<a href="https://colab.research.google.com/github/camilachavarriaga2001-hash/Integracion_de_datos-/blob/main/Reto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Caso de estudio:**


El trabajo desarrollado tuvo como propósito aplicar el método de agrupamiento K-Medoids a la base de datos objeto de estudio, con el fin de identificar patrones y segmentar los registros en grupos homogéneos. A través de esta técnica, se buscó analizar las características de la población desde una perspectiva estadística y facilitar la interpretación de la información mediante la conformación de clústeres representativos. De esta manera, el análisis permitió obtener una visión más clara de los datos, reduciendo la complejidad y ofreciendo insumos relevantes para la generación de conclusiones más precisas.

La base de datos utilizada contiene las siguientes variables:

* ID :Identificador único asignado a cada registro en la base de datos.

* Warehouse_block:Bloque del almacén desde el cual se despacha el producto.

* Mode_of_Shipment:Medio de transporte utilizado para el envío, como barco, vuelo o carretera.

* Customer_care_calls:Número de llamadas realizadas por el cliente al área de servicio de atención.

* Customer_rating:Calificación otorgada por el cliente respecto al servicio recibido.

* Cost_of_the_Product:Costo asociado al producto adquirido en la transacción.

* Prior_purchases:Número de compras realizadas previamente por el cliente.

* Product_importance:Nivel de importancia asignado al producto, clasificado en categorías como bajo, medio o alto.

* Gender:Género del cliente registrado en la transacción.

* Discount_offered:Descuento aplicado al producto en la compra.

* Weight_in_gms:Peso del producto expresado en gramos.

* Reached.on.Time_Y.N :Indica si el envío llegó a tiempo o se retras.



Para dar cumplimiento al objetivo planteado, se llevó a cabo la aplicación del método K-Medoids y la correspondiente evaluación de las variables, considerando diferentes métricas que permitieron valorar la calidad de la segmentación y la caracterización de los clústeres obtenidos.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nxl = '/content/drive/MyDrive/Colab Notebooks/Integracion y prospectiva /Train.csv'
XDB=pd.read_csv(nxl)
display(XDB)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [ ]:
numeric_XDB = XDB.select_dtypes(include=np.number)
correlations_matrix = numeric_XDB.corr()
display(correlations_matrix)

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,1.000000,0.188998,-0.005722,0.196791,0.145369,-0.598278,0.278312,-0.411822
Customer_care_calls,0.188998,1.000000,0.012209,0.323182,0.180771,-0.130750,-0.276615,-0.067126
Customer_rating,-0.005722,0.012209,1.000000,0.009270,0.013179,-0.003124,-0.001897,0.013119
Cost_of_the_Product,0.196791,0.323182,0.009270,1.000000,0.123676,-0.138312,-0.132604,-0.073587
Prior_purchases,0.145369,0.180771,0.013179,0.123676,1.000000,-0.082769,-0.168213,-0.055515
Discount_offered,-0.598278,-0.130750,-0.003124,-0.138312,-0.082769,1.000000,-0.376067,0.397108
Weight_in_gms,0.278312,-0.276615,-0.001897,-0.132604,-0.168213,-0.376067,1.000000,-0.268793
Reached.on.Time_Y.N,-0.411822,-0.067126,0.013119,-0.073587,-0.055515,0.397108,-0.268793,1.000000


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

df = XDB.copy() # Use the existing XDB DataFrame

quant_vars = [
    "Customer_care_calls",
    "Customer_rating",
    "Cost_of_the_Product",
    "Prior_purchases",
    "Discount_offered",
    "Weight_in_gms",
]

qual_vars = [
    "Warehouse_block",
    "Mode_of_Shipment",
    "Product_importance",
    "Gender",
]

quant_results = []
for block, subset in df.groupby("Warehouse_block"):
    for var in quant_vars:
        s = pd.to_numeric(subset[var], errors="coerce").dropna()
        quant_results.append({
            "Warehouse_block": block,
            "Variable": var,
            "N": s.size,
            "Media": s.mean(),
            "Desviación estándar": s.std(ddof=1),
            "Asimetría": skew(s, bias=False) if s.size > 2 else np.nan,
            "Curtosis": kurtosis(s, bias=False) if s.size > 3 else np.nan,
        })
quant_df = pd.DataFrame(quant_results).sort_values(["Warehouse_block","Variable"]).reset_index(drop=True)

qual_results = []
for block, subset in df.groupby("Warehouse_block"):
    for var in qual_vars:
        freqs = subset[var].value_counts(normalize=True, dropna=False) * 100
        for cat, pct in freqs.items():
            qual_results.append({
                "Warehouse_block": block,
                "Variable": var,
                "Categoría": cat if pd.notna(cat) else "NaN",
                "Porcentaje": round(pct, 2),
            })
qual_df = pd.DataFrame(qual_results).sort_values(["Warehouse_block","Variable","Categoría"]).reset_index(drop=True)

quant_df.head(), qual_df.head()

(  Warehouse_block             Variable     N       Media  Desviación estándar  \
 0               A  Cost_of_the_Product  1833  208.767594            48.623021   
 1               A  Customer_care_calls  1833    4.038189             1.138673   
 2               A      Customer_rating  1833    2.957447             1.434654   
 3               A     Discount_offered  1833   13.222586            16.214130   
 4               A      Prior_purchases  1833    3.577741             1.515418   
 
    Asimetría  Curtosis  
 0  -0.133381 -0.998955  
 1   0.411495 -0.318231  
 2   0.045803 -1.334819  
 3   1.852716  2.191755  
 4   1.659303  3.901292  ,
   Warehouse_block          Variable Categoría  Porcentaje
 0               A            Gender         F       50.63
 1               A            Gender         M       49.37
 2               A  Mode_of_Shipment    Flight       16.20
 3               A  Mode_of_Shipment      Road       16.04
 4               A  Mode_of_Shipment      Ship       6

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

df = XDB.copy()  # Usamos el DataFrame original

# Variables cuantitativas y cualitativas
quant_vars = [
    "Customer_care_calls",
    "Customer_rating",
    "Cost_of_the_Product",
    "Prior_purchases",
    "Discount_offered",
    "Weight_in_gms",
]

qual_vars = [
    "Warehouse_block",
    "Mode_of_Shipment",
    "Product_importance",
    "Gender",
]

# --- SOLO bloque B ---
df_b = df[df["Warehouse_block"] == "B"]

# Cuantitativas
quant_results = []
for var in quant_vars:
    s = pd.to_numeric(df_b[var], errors="coerce").dropna()
    quant_results.append({
        "Warehouse_block": "B",
        "Variable": var,
        "N": s.size,
        "Media": s.mean(),
        "Desviación estándar": s.std(ddof=1),
        "Asimetría": skew(s, bias=False) if s.size > 2 else np.nan,
        "Curtosis": kurtosis(s, bias=False) if s.size > 3 else np.nan,
    })
quant_df_b = pd.DataFrame(quant_results).sort_values("Variable").reset_index(drop=True)

# Cualitativas
qual_results = []
for var in qual_vars:
    freqs = df_b[var].value_counts(normalize=True, dropna=False) * 100
    for cat, pct in freqs.items():
        qual_results.append({
            "Warehouse_block": "B",
            "Variable": var,
            "Categoría": cat if pd.notna(cat) else "NaN",
            "Porcentaje": round(pct, 2),
        })
qual_df_b = pd.DataFrame(qual_results).sort_values(["Variable","Categoría"]).reset_index(drop=True)

quant_df_b.head(), qual_df_b.head()


(  Warehouse_block             Variable     N       Media  Desviación estándar  \
 0               B  Cost_of_the_Product  1833  212.159302            46.980005   
 1               B  Customer_care_calls  1833    4.020185             1.123939   
 2               B      Customer_rating  1833    2.980360             1.416391   
 3               B     Discount_offered  1833   13.187125            15.818064   
 4               B      Prior_purchases  1833    3.575014             1.531677   
 
    Asimetría  Curtosis  
 0  -0.231631 -0.858180  
 1   0.350561 -0.297958  
 2   0.030108 -1.299168  
 3   1.823545  2.153206  
 4   1.671474  3.957014  ,
   Warehouse_block          Variable Categoría  Porcentaje
 0               B            Gender         F       49.54
 1               B            Gender         M       50.46
 2               B  Mode_of_Shipment    Flight       16.15
 3               B  Mode_of_Shipment      Road       16.04
 4               B  Mode_of_Shipment      Ship       6

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

df = XDB.copy()  # Usamos el DataFrame original

# Variables cuantitativas y cualitativas
quant_vars = [
    "Customer_care_calls",
    "Customer_rating",
    "Cost_of_the_Product",
    "Prior_purchases",
    "Discount_offered",
    "Weight_in_gms",
]

qual_vars = [
    "Warehouse_block",
    "Mode_of_Shipment",
    "Product_importance",
    "Gender",
]

# --- SOLO bloque C ---
df_c = df[df["Warehouse_block"] == "C"]

# Cuantitativas
quant_results = []
for var in quant_vars:
    s = pd.to_numeric(df_c[var], errors="coerce").dropna()
    quant_results.append({
        "Warehouse_block": "C",
        "Variable": var,
        "N": s.size,
        "Media": s.mean(),
        "Desviación estándar": s.std(ddof=1),
        "Asimetría": skew(s, bias=False) if s.size > 2 else np.nan,
        "Curtosis": kurtosis(s, bias=False) if s.size > 3 else np.nan,
    })
quant_df_c = pd.DataFrame(quant_results).sort_values("Variable").reset_index(drop=True)

# Cualitativas
qual_results = []
for var in qual_vars:
    freqs = df_c[var].value_counts(normalize=True, dropna=False) * 100
    for cat, pct in freqs.items():
        qual_results.append({
            "Warehouse_block": "C",
            "Variable": var,
            "Categoría": cat if pd.notna(cat) else "NaN",
            "Porcentaje": round(pct, 2),
        })
qual_df_c = pd.DataFrame(qual_results).sort_values(["Variable","Categoría"]).reset_index(drop=True)

quant_df_c.head(), qual_df_c.head()


(  Warehouse_block             Variable     N       Media  Desviación estándar  \
 0               C  Cost_of_the_Product  1833  211.191489            47.421596   
 1               C  Customer_care_calls  1833    4.064921             1.145352   
 2               C      Customer_rating  1833    2.993999             1.403157   
 3               C     Discount_offered  1833   13.399891            16.279322   
 4               C      Prior_purchases  1833    3.554828             1.525393   
 
    Asimetría  Curtosis  
 0  -0.123784 -0.999789  
 1   0.363847 -0.373030  
 2  -0.000012 -1.276505  
 3   1.770258  1.883125  
 4   1.725362  4.180939  ,
   Warehouse_block          Variable Categoría  Porcentaje
 0               C            Gender         F       50.25
 1               C            Gender         M       49.75
 2               C  Mode_of_Shipment    Flight       16.09
 3               C  Mode_of_Shipment      Road       16.04
 4               C  Mode_of_Shipment      Ship       6

In [ ]:
np.random.seed(42)
K_b = 3  # Assuming 3 clusters for block B as well
Xmin_b, Xmax_b = np.min(XD_b, axis=0), np.max(XD_b, axis=0)

XC_b = np.zeros((K_b, XD_b.shape[1]))
for j in range(K_b):
    XC_b[j,:] = Xmin_b + (Xmax_b - Xmin_b) * np.random.random()
    if j < len(XD_b):
        XC_b[j,:] = XD_b[j,:]

centros_b_init = pd.DataFrame(XC_b, columns=quant_vars).assign(Cluster=[str(i) for i in range(K_b)])
display(centros_b_init)

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Cluster
0,3.0,3.0,176.0,4.0,10.0,1177.0,0
1,3.0,2.0,164.0,3.0,29.0,1187.0,1
2,4.0,3.0,227.0,3.0,45.0,2707.0,2


In [ ]:
quant_vars = [
    "Customer_care_calls","Customer_rating","Cost_of_the_Product",
    "Prior_purchases","Discount_offered","Weight_in_gms"
]
y_col = "Reached.on.Time_Y.N"

ref_blocks  = ["A","B","C"]   # referencia
inte_blocks = ["C","D","F"]   # a integrar (no hay 'E' en esta base)

XDB_ref = df[df["Warehouse_block"].isin(ref_blocks)].copy()
XD  = np.array(XDB_ref[quant_vars], dtype=float)
yd  = np.array(XDB_ref[y_col], dtype=int)

XD.shape, np.bincount(pd.factorize(XDB_ref["Warehouse_block"])[0])


((5499, 6), array([1833, 1833, 1833]))

In [ ]:
np.random.seed(42)
K = 3
Xmin, Xmax = np.min(XD, axis=0), np.max(XD, axis=0)

XC = np.zeros((K, XD.shape[1]))
for j in range(K):
    XC[j,:] = Xmin + (Xmax - Xmin) * np.random.random()
    if j < len(XD):                # como en tu ejemplo: usar los j primeros para arrancar
        XC[j,:] = XD[j,:]

pd.DataFrame(XC, columns=quant_vars).assign(Cluster=["0","1","2"])


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Cluster
0,2.0,2.0,183.0,4.0,48.0,3374.0,0
1,3.0,3.0,176.0,4.0,10.0,1177.0,1
2,2.0,2.0,184.0,3.0,46.0,2484.0,2


In [ ]:
fhat = np.zeros((len(XD), 1), dtype=int)

for k in range(len(XD)):
    VP = np.exp(-0.5 * (np.mean(((XC[:] - XD[k,:]) / (XC[:] + 1e-9))**2, axis=1)))
    nc = int(np.argmax(VP))
    fhat[k,0] = nc
    XC[nc,:] = (XC[nc,:] + XD[k,:]) / 2.0

centros_ref = pd.DataFrame(XC, columns=quant_vars)
centros_ref.index = [f"Bloque_{i}(Ref)" for i in range(K)]
centros_ref


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Bloque_0(Ref),4.210800,1.000005,247.370083,5.014613,2.454613,1473.719013
Bloque_1(Ref),4.281636,1.321778,249.687778,5.124962,1.001793,1362.337825
Bloque_2(Ref),4.708323,2.971679,234.119363,5.090363,4.707800,1234.461184


In [ ]:
XDB_int = df[df["Warehouse_block"].isin(inte_blocks)].copy()
XD2 = np.array(XDB_int[quant_vars], dtype=float)

XC2 = np.copy(XC)   # copia de centros antes de integrar
for k in range(len(XD2)):
    VP2 = np.exp(-0.5 * (np.mean(((XC2[:] - XD2[k,:]) / (XC2[:] + 1e-9))**2, axis=1)))
    nc2 = int(np.argmax(VP2))
    # print(f"Registro {k} → bloque {nc2}")  # descomenta si quieres ver cada asignación
    XC2[nc2,:] = (XC2[nc2,:] + XD2[k,:]) / 2.0

centros_post = pd.DataFrame(XC2, columns=quant_vars)
centros_post.index = [f"Bloque_{i}(Post)" for i in range(K)]
centros_post


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Bloque_0(Post),4.889998,2.084420,236.236189,5.876725,1.864432,1242.701046
Bloque_1(Post),4.827471,1.000032,254.890481,5.579377,1.216361,1210.790993
Bloque_2(Post),3.490682,3.974610,198.302591,5.033211,5.294981,1506.994770


In [ ]:
orig = pd.DataFrame(XC,  columns=quant_vars); orig.index = [f"Bloque_{i}" for i in range(K)]
post = pd.DataFrame(XC2, columns=quant_vars); post.index = [f"Bloque_{i}" for i in range(K)]

delta_pct = (np.abs((XC - XC2) / (XC + 1e-9))) * 100
delta_pct = pd.DataFrame(delta_pct, columns=quant_vars, index=[f"Bloque_{i}" for i in range(K)])

print("Centros referencia (finales del paso 4):"); display(orig)
print("Centros post-integración (tras paso 6):");  display(post)
print("Cambio porcentual absoluto (%):");          display(delta_pct.round(2))


Centros referencia (finales del paso 4):


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Bloque_0,4.210800,1.000005,247.370083,5.014613,2.454613,1473.719013
Bloque_1,4.281636,1.321778,249.687778,5.124962,1.001793,1362.337825
Bloque_2,4.708323,2.971679,234.119363,5.090363,4.707800,1234.461184


Centros post-integración (tras paso 6):


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Bloque_0,4.889998,2.084420,236.236189,5.876725,1.864432,1242.701046
Bloque_1,4.827471,1.000032,254.890481,5.579377,1.216361,1210.790993
Bloque_2,3.490682,3.974610,198.302591,5.033211,5.294981,1506.994770


Cambio porcentual absoluto (%):


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
Bloque_0,16.13,108.44,4.50,17.19,24.04,15.68
Bloque_1,12.75,24.34,2.08,8.87,21.42,11.12
Bloque_2,25.86,33.75,15.30,1.12,12.47,22.08


In [ ]:
X_all = np.array(df[quant_vars], dtype=float)
VP_all = []
for k in range(len(X_all)):
    VP2 = np.exp(-0.5 * (np.mean(((XC2[:] - X_all[k,:]) / (XC2[:] + 1e-9))**2, axis=1)))
    VP_all.append(VP2)
VP_all = np.vstack(VP_all)
asig_all = np.argmax(VP_all, axis=1)

df_post = df.copy()
df_post["Bloque_integrado_idx"] = asig_all
map_idx_to_block = {0:"A", 1:"B", 2:"C"}   # etiqueta A/B/C por índice
df_post["Bloque_integrado"] = df_post["Bloque_integrado_idx"].map(map_idx_to_block)
df_post[["Warehouse_block","Bloque_integrado"] + quant_vars].head()


,Warehouse_block,Bloque_integrado,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
0,D,C,4,2,177,3,44,1233
1,F,C,4,5,216,2,59,3088
2,A,C,2,2,183,4,48,3374
3,B,C,3,3,176,4,10,1177
4,C,C,2,2,184,3,46,2484


In [ ]:
def stats_s(s):
    s = pd.to_numeric(s, errors="coerce").dropna()
    return pd.Series({
        "N": s.size,
        "Media": s.mean(),
        "Desviación": s.std(ddof=1),
        "Asimetría": skew(s, bias=False) if s.size > 2 else np.nan,
        "Curtosis": kurtosis(s, bias=False) if s.size > 3 else np.nan
    })

quant_char = []
for b, sub in df_post.groupby("Bloque_integrado"):
    for v in quant_vars:
        r = stats_s(sub[v]); r.name = None
        quant_char.append({"Bloque": b, "Variable": v, **r.to_dict()})

quant_char_df = pd.DataFrame(quant_char).sort_values(["Bloque","Variable"]).reset_index(drop=True)
quant_char_df.head(12)


/tmp/ipython-input-2922030384.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Asimetría": skew(s, bias=False) if s.size > 2 else np.nan,
/tmp/ipython-input-2922030384.py:8: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Curtosis": kurtosis(s, bias=False) if s.size > 3 else np.nan


,Bloque,Variable,N,Media,Desviación,Asimetría,Curtosis
0,A,Cost_of_the_Product,333.0,250.933934,43.072947,-1.240142,1.291364
1,A,Customer_care_calls,333.0,5.279279,1.142059,-0.539209,0.171708
2,A,Customer_rating,333.0,2.171171,0.880244,0.192077,-0.820956
3,A,Discount_offered,333.0,1.984985,0.774062,0.104287,-1.116329
4,A,Prior_purchases,333.0,4.348348,0.962539,-0.562448,0.494948
5,A,Weight_in_gms,333.0,1480.111111,293.122733,0.100143,-1.134810
6,B,Cost_of_the_Product,45.0,242.933333,52.004108,-1.392084,1.188881
7,B,Customer_care_calls,45.0,4.688889,1.239909,0.105931,-1.024213
8,B,Customer_rating,45.0,1.000000,0.000000,NaN,NaN
9,B,Discount_offered,45.0,1.000000,0.000000,NaN,NaN


In [ ]:
qual_vars = ["Mode_of_Shipment","Product_importance","Gender"]

qual_char = []
for b, sub in df_post.groupby("Bloque_integrado"):
    for v in qual_vars:
        freqs = sub[v].value_counts(normalize=True, dropna=False).mul(100)
        for cat, pct in freqs.items():
            qual_char.append({"Bloque": b, "Variable": v, "Categoría": cat if pd.notna(cat) else "NaN", "Porcentaje": round(pct,2)})

qual_char_df = pd.DataFrame(qual_char).sort_values(["Bloque","Variable","Categoría"]).reset_index(drop=True)
qual_char_df.head(12)


,Bloque,Variable,Categoría,Porcentaje
0,A,Gender,F,53.15
1,A,Gender,M,46.85
2,A,Mode_of_Shipment,Flight,15.92
3,A,Mode_of_Shipment,Road,15.32
4,A,Mode_of_Shipment,Ship,68.77
5,A,Product_importance,high,1.50
6,A,Product_importance,low,58.86
7,A,Product_importance,medium,39.64
8,B,Gender,F,42.22
9,B,Gender,M,57.78


In [ ]:
with pd.ExcelWriter("integracion_y_caracterizacion.xlsx") as w:
    tabla_ref.to_excel(w, sheet_name="Clusters_referencia", index=True)
    centros_post.to_excel(w, sheet_name="Clusters_post", index=True)
    delta_pct.round(2).to_excel(w, sheet_name="Cambios_%", index=True)
    quant_char_df.to_excel(w, sheet_name="Cuantitativas_post", index=False)
    qual_char_df.to_excel(w, sheet_name="Cualitativas_post", index=False)

"Archivo: integracion_y_caracterizacion.xlsx"


'Archivo: integracion_y_caracterizacion.xlsx'

# **Analisis ejecutivo:**

El análisis realizado a la base de datos muestra los siguientes resultados:

En cuanto a la matriz de correlación, se observa que existe una relación relativamente alta entre el costo del producto y el número de llamadas realizadas por el cliente al área de servicio. Asimismo, se identifica una correlación significativa entre el número de compras previas y el número de llamadas al área de servicio.

* **análisis de métricas en el bloque A **: se encontró que la mayoría de las asimetrías de las variables se ubican por debajo del 1%, lo cual refleja un comportamiento adecuado. Sin embargo, las medias de las variables se encuentran muy por encima de este valor. En el caso de la curtosis, la mayoría de las variables presentan un desempeño desfavorable, con excepción de Prior_purchases, que es la que más se aproxima al valor esperado del 5%, alcanzando aproximadamente un 4%. La desviación estándar, por su parte, muestra valores que en algunos casos se encuentran muy por encima o muy por debajo del 5% esperado.

* **análisis de métricas en el bloque B **: se encontró que la mayoría de las asimetrías de las variables se ubican por debajo del 1%, lo cual refleja un comportamiento adecuado. Sin embargo, las medias de las variables se encuentran muy por encima de este valor. En el caso de la curtosis, la mayoría de las variables presentan un desempeño desfavorable, con excepción de Prior_purchases, que es la que más se aproxima al valor esperado del 5%, alcanzando aproximadamente un 4%. La desviación estándar, por su parte, muestra valores que en algunos casos se encuentran muy por encima o muy por debajo del 5% esperado.

* **análisis de métricas en el bloque C**: se encontró que la mayoría de las asimetrías de las variables se ubican por debajo del 1%, lo cual refleja un comportamiento adecuado. Sin embargo, las medias de las variables se encuentran muy por encima de este valor. En el caso de la curtosis, la mayoría de las variables presentan un desempeño desfavorable, con excepción de Prior_purchases, que es la que más se aproxima al valor esperado del 5%, alcanzando aproximadamente un 4%. La desviación estándar, por su parte, muestra valores que en algunos casos se encuentran muy por encima o muy por debajo del 5% esperado.


Finalmente, al aplicar la segmentación, se evidenció que el porcentaje de cambio en la mayoría de las variables no es muy elevado, lo cual sugiere una buena integración de los datos. No obstante, se detectó un comportamiento atípico en la variable Customer_rating dentro del bloque 0, que presentó una variación negativa del 108%.


**Conclusión**

El análisis evidenció correlaciones relevantes entre el costo del producto, las compras previas y las llamadas al área de servicio. Las métricas mostraron asimetrías adecuadas, aunque con diferencias en medias, curtosis y desviaciones estándar. La segmentación indicó una integración consistente de los datos y comportamientos semejantes entre variables, salvo en Customer_rating del bloque 0, donde se presentó una variación negativa significativa que requiere mayor análisis.
